In [ ]:
import urx
import math
import numpy as np
import time
import plotly.graph_objects as go
from numpy import *

In [ ]:
def JointLocations(thetas):
    
    d1 = 0.1625
    a2 = -0.425
    a3 = -0.3922
    d4 = 0.1333
    d5 = 0.0997 
    d6 = 0.0996 
    
    t1 = thetas[0]
    t2 = thetas[1]
    t3 = thetas[2]
    t4 = thetas[3]
    t5 = thetas[4]
    t23 = t2 +  t3
    t234 = t2 + t3 + t4
    
    theta1 = [0,0,d1]
    
    theta2 = [(a2*np.cos(t1)*np.cos(t2)),
               (a2*np.cos(t2)*np.sin(t1)),
               (d1+(a2*np.sin(t2)))]
               
    theta3 = [np.cos(t1)*((a2*np.cos(t2)) + (a3*np.cos(t23))),
              ((a2*np.cos(t2)) + (a3*np.cos(t23))) *np.sin(t1),
              d1 + (a2*np.sin(t2))+(a3*np.sin(t23))]
               
    theta4 = [(np.cos(t1)*(a2*np.cos(t2)+a3*np.cos(t23)) + d4*np.sin(t1)),
              -d4*np.cos(t1) + ((a2*np.cos(t2)) + (a3*np.cos(t23)))*np.sin(t1),
              d1 + a2*np.sin(t2) + a3*np.sin(t23)]
               
    theta5 = [ d4*np.sin(t1) + (np.cos(t1) * ((a2*np.cos(t2)) + (a3*np.cos(t23)) + (d5*np.sin(t234)))),
              -d4*np.cos(t1) + (np.sin(t1) * ((a2*np.cos(t2)) + (a3*np.cos(t23)) + (d5*np.sin(t234)))),
              d1 - (d5*np.cos(t234)) + (a2*np.sin(t2)) + (a3*np.sin(t23))]
               
    theta6 = [((d4+(d6*np.cos(t5)))*np.sin(t1)) + np.cos(t1) * ((a2*np.cos(t2)) + (a3*np.cos(t23)) + (d5*np.sin(t234)) -(d6*np.cos(t234)*np.sin(t5))),
              (-np.cos(t1) * (d4+ (d6*np.cos(t5)))) + np.sin(t1) * ((a2*np.cos(t2)) + (a3*np.cos(t23)) + (d5*np.sin(t234)) - (d6*np.cos(t234)*np.sin(t5))),
              d1 - (d5*np.cos(t234)) + (a2*np.sin(t2)) + (a3*np.sin(t23)) - (d6*np.sin(t234)*np.sin(t5))]
    
    positions = [theta1,theta2,theta3,theta4,theta5,theta6]
    return positions

def Vec2UnitVec(vec):
    norm = np.sqrt( np.square(vec[0]) + np.square(vec[1]) + np.square(vec[2]))
    vector = vec / norm
    return vector

def VecNorm(vec):
    norm = np.sqrt( np.square(vec[0]) + np.square(vec[1]) + np.square(vec[2]))
    return norm

In [7]:
robo = urx.Robot("172.16.101.225")
# robo = urx.Robot("127.0.0.1")

tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 44
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 68
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1092
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1454


In [ ]:
while True: #Elbow Singularity
    fig = go.Figure()
    x = []
    y = []
    z = []
    
#     Positions = JointLocations([0,-np.pi/2,np.pi/2,0,np.pi/2,np.pi])
    Positions = JointLocations(robo.getj())
    wristvec1 = np.asarray(Positions[3]) - np.asarray(Positions[2])
    wristvec2 = np.asarray(Positions[5]) - np.asarray(Positions[4])
    Resultatnt = np.cross(Vec2UnitVec( wristvec1), Vec2UnitVec( wristvec2))
    print(VecNorm(Resultatnt),"Wrist")
        
    wristvec1 = np.asarray(Positions[1]) - np.asarray(Positions[0])
    wristvec2 = np.asarray(Positions[2]) - np.asarray(Positions[1])
    Resultatnt1 = np.cross(Vec2UnitVec( wristvec1), Vec2UnitVec( wristvec2))
    print(VecNorm(Resultatnt1),"Elbow")
    
    
#     wristvec1 = np.asarray(Positions[1]) - np.asarray(Positions[0])
#     wristvec3 = np.asarray(Positions[3]) - np.asarray(Positions[2])

#     wristvec2 = np.asarray(Positions[5]) - np.asarray(Positions[3])
# #     res = np.cross(Vec2UnitVec( wristvec2), Vec2UnitVec( wristvec3))
#     Resultatnt2 = np.cross(Vec2UnitVec( wristvec1), Vec2UnitVec( wristvec2))
#     print(VecNorm(Resultatnt2),"Shoulder")
    
    p1 = Positions[0]
    p2 = [0,0.1333,0.1625]
    vector1 = np.subtract(p2,p1)
    p = p1 + 0.5*(vector1)
    p = [p[0],p[1],p[2]+10]
    vec2 = np.subtract(p2,p)

    a,b,c = np.cross(vec2,vector1)
    d = a*p[0] + b*p[1] + c*p[2]
    d1 = a* Positions[2][0] + b*Positions[2][1] + c*Positions[2][2]
    d2 = a* Positions[3][0] + b*Positions[3][1] + c*Positions[3][2]
    print(d)
    print(d1)
    print(d2)
    for i in range(len(Positions)):
        x.append(Positions[i][0])
        y.append(Positions[i][1])
        z.append(Positions[i][2])
    if (d1 <= d+0.2 and d1 >= d-0.2) and (d2 <= d+0.2 and d2 >= d-0.2): 
        print("Shoulder")
    data = go.Scatter3d(x=x, y=y,z=z)
    data1 = go.Scatter3d(x=[Positions[0][0],p2[0]],y=[Positions[0][1],p2[1]],z=[Positions[0][2],p2[2]])
    fig.add_trace(data)
    fig.add_trace(data1)
    fig.show()   
    break
    

In [ ]:
## shoulder 
# robo.movej([-0.06977492967714483, -2.534757753411764, 2.05109674135317, 0.483809157008789, 1.5024313926696777, 3.1407737731933594],0.2,0.2,wait=False)
robo.movel([-0.10705350142130086, -0.13283605224840636, 0.4879043453543332, -1.2114441235423856, -1.2078457149245494, 1.2092390087185882],0.2,0.2,wait=False)


In [ ]:
## elbow

robo.movej([0.7972707748413086,
 -0.5574820798686524,
 0.02177602449526006,
 0.5355483728596191,
 2.368805408477783,
 3.143150806427002],0.2,0.2,wait=False)

In [ ]:
## wrist
# robo.movej([-1.3761499563800257, -0.4601392310908814, 1.8812263647662562, -4.30295004467153, -0.5618003050433558, 5.4686055183410645],0.2,0.2,wait=False)
robo.movel([-0.3126986484064443, 0.46471863658009055, 0.04567248674201419, -1.068165826146162, -1.3611541178786069, 0.46179122711978055],0.2,0.2,wait=False)

In [40]:
#Home position

robo.movej(( 0,-math.pi/2,math.pi/2,0,math.pi/2,math.pi),0.4,0.4,wait=False)

Exception in thread Thread-19:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/shivendra/virtual_robotics_urx/lib/python3.5/site-packages/urx/ursecmon.py", line 286, in run
    data = self._get_data()
  File "/home/shivendra/virtual_robotics_urx/lib/python3.5/site-packages/urx/ursecmon.py", line 333, in _get_data
    tmp = self._s_secondary.recv(1024)
ConnectionResetError: [Errno 104] Connection reset by peer



In [ ]:
# robo.movej([0.028643131256103516,
#  -2.4621011219420375,
#  1.8871234099017542,
#  0.5666495996662597,
#  1.803328037261963,
#  2.9882988929748535],0.2,0.2,wait=False)

In [ ]:
current_value_l = []
current_value_j = []
while robo.is_program_running():
    c = robo.getl()
    d = robo.getj()
    current_value_l.append(c)
    current_value_j.append(d)
    time.sleep(1)
print(current_value_l)
print(current_value_j)
print(len(current_value_j))

In [ ]:
jv =[[-0.11755825002816334, -1.588956108050498, 1.5886952059416677, 0.0002609021088302299, 1.235712176989893, 3.141592653589793],
[-0.11735838035549917, -1.6322636271396274, 1.6671200328209412, -0.034219952804795284, 1.4547193402539726, 3.1432282238192513],
[-0.19623702544099597, -1.657019553818711, 1.7073410644023728, 0.027898477786155063, 1.305201702151999, 3.0982207670132005],
[-0.23693623276479947, -1.6835041052044648, 1.761448588817231, 0.04191597742157516, 1.339703827092271, 3.071354135434561],
[-0.27693214535859045, -1.7041717687323557, 1.813165981264886, 0.053302378691003405, 1.3757139352972885, 3.0414533957235097],
[-0.31480378684948085, -1.7183284820167364, 1.8631391410344147, 0.05998006653564758, 1.4146475650402175, 3.0091984261846587],
[-0.34918873009485707, -1.7252185428136197, 1.9117982121836556, 0.059953138793679805, 1.4577997463826102, 2.9754675316463652],
[-0.3789021589176311, -1.7239724347738346, 1.9593113145346877, 0.05131416615141582, 1.506220629359391, 2.9412019626896346],
[-0.403087594664725, -1.7135909620303815, 2.005539110127854, 0.03230435989443414, 1.5605739840516542, 2.9072381899710242],
[-0.4526287992770628, -1.7082132588432088, 2.069128025058707, -0.01909630177342958, 1.552677360710728, 2.87726758629572],
[-0.5302062032468808, -1.7049884852985286, 2.147584369998844, -0.09935115023856955, 1.4796024790608275, 2.851178289925935],
[-0.6047302166812958, -1.686463498603132, 2.220703336072268, -0.18779565426046396, 1.4094620417435528, 2.8257884564809714],
[-0.67566459714971, -1.6508160016858238, 2.2877197299453083, -0.28567717308057405, 1.3427964826486174, 2.801080620973202],
[-0.742913056696626, -1.5963801974361722, 2.347477860063791, -0.39367341465301253, 1.2797222830277253, 2.7769437052079553],
[-0.8068495576195243, -1.5221187221873578, 2.3984495360610407, -0.5113920087720172, 1.2199080026113784, 2.7531372713102824],
[-0.8681222516988865, -1.4282015309712133, 2.438850120795679, -0.6368858074740591, 1.1627645365945023, 2.7293263535765804],
[-0.9275812224789415, -1.316750487641043, 2.466852127286743, -0.7660985067642265, 1.107519541772895, 2.705070565566399]]

In [ ]:
print(robo.getj())

In [ ]:
Positions = JointLocations(jv[1])
wristvec1 = np.asarray(Positions[1]) - np.asarray(Positions[0])
wristvec2 = np.asarray(Positions[5]) - np.asarray(Positions[3])
Resultatnt = np.cross(Vec2UnitVec( wristvec1), Vec2UnitVec( wristvec2))
print(Resultatnt)


In [ ]:
print(robo.getl())

In [ ]:
def Singularity(jv,step):
    stack = []
    jv1 = []
    arr1 = []
    for i in range(len(jv)-1):
        mid = np.linspace(jv[i],jv[i+1],step)
        arr1.append(mid)
        for j in range(len(arr1[0])):
            jv1.append(list(arr1[i][j]))
    for i in range(len(jv1)):        
        Positions = JointLocations(jv1[i])
    #     print(Positions)
        wristvec1 = np.asarray(Positions[3]) - np.asarray(Positions[2])
        wristvec2 = np.asarray(Positions[5]) - np.asarray(Positions[4])
        Resultatnt = np.cross(Vec2UnitVec( wristvec1), Vec2UnitVec( wristvec2))
        Res = VecNorm(Resultatnt)#wrist
        elbowvec1 = np.asarray(Positions[1]) - np.asarray(Positions[0])
        elbowvec2 = np.asarray(Positions[2]) - np.asarray(Positions[1])
        Resultatnt1 = np.cross(Vec2UnitVec( elbowvec1), Vec2UnitVec( elbowvec2))
        Res1 = VecNorm(Resultatnt1)#elbow
        
        p1 = Positions[0]
        p2 = [0,0.1333,0.1625]
#         p2 = [0,-Positions[3][1],p1[2]]
        vector1 = np.subtract(p2,p1)
        p = p1 + 0.5*(vector1)
        p = [p[0],p[1],p[2]+10]
        vec2 = np.subtract(p2,p)

        a,b,c = np.cross(vec2,vector1)
        d = a*p[0] + b*p[1] + c*p[2]
        d1 = a* Positions[2][0] + b*Positions[2][1] + c*Positions[2][2]
        d2 = a* Positions[3][0] + b*Positions[3][1] + c*Positions[3][2]
        if (d1 <= d+0.2 and d1 >= d-0.2) and (d2 <= d+0.2 and d2 >= d-0.2): 
            print("Shoulder")
            retval = 1
            stack.append(retval)
        elif Res>=0 and Res<=0.2:
            print(Res,"Wrist",i)
            retval = 1
            stack.append(retval)
        elif Res1>=0 and Res1<=0.2:
            print(Res1,"Elbow",i)
            retval = 1
            stack.append(retval)
        else:
            print("NO SINGULARITY")
            retval = 0
            stack.append(retval)
    if np.any(stack) == 1:
        return 1
    else: 
        return 0

In [ ]:
Singularity(jv,5)

In [ ]:
def a(jv,dist):
    arr1 = []
    jv1 = []
    for i in range(len(jv)-1):
        arr = np.linspace(jv[i],jv[i+1],dist)
        arr1.append(arr)
        for j in range(len(arr1[0])):
            jv1.append(list(arr1[i][j]))
#         jv = arr1
#         for j in range(len(arr[0])):
#             jv.insert(i,arr1[i][j])
    print(jv1)
#     print(jv)
    print(len(jv1))
#     print(jv)

In [ ]:
a(jv,5)

In [ ]:
jv = [[-1.0328727426480577, 4.711942876314112, -4.262348083154811, -0.44745266165273456, 2.633501137594452, 0.0009163376558587007]]

In [ ]:
for i in range(len(jv)):
    for j in range(len(jv[0])):
        if jv[i][j] >= 3:
            jv[i][j] = -2*pi + jv[i][j]
        if jv[i][j] <= -3:
            jv[i][j] = 2*pi + jv[i][j]
print(jv)

In [10]:
robo.set_freedrive(1)

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/shivendra/virtual_robotics_urx/lib/python3.5/site-packages/urx/ursecmon.py", line 286, in run
    data = self._get_data()
  File "/home/shivendra/virtual_robotics_urx/lib/python3.5/site-packages/urx/ursecmon.py", line 333, in _get_data
    tmp = self._s_secondary.recv(1024)
ConnectionResetError: [Errno 104] Connection reset by peer
Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/shivendra/virtual_robotics_urx/lib/python3.5/site-packages/urx/ursecmon.py", line 286, in run
    data = self._get_data()
  File "/home/shivendra/virtual_robotics_urx/lib/python3.5/site-packages/urx/ursecmon.py", line 333, in _get_data
    tmp = self._s_secondary.recv(1024)
ConnectionResetError: [Errno 104] Connection reset by peer


In [41]:
robo = urx.Robot("172.16.101.225")
# robo = urx.Robot("127.0.0.1")

tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 68
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1092
tried 11 times to find a packet in data, advertised packet size: -2, type: 3
Data length: 1454


In [20]:
robo.getj()

[0.009172797203063965,
 -0.8363850873759766,
 1.953855339680807,
 -1.1332073968699952,
 1.5791306495666504,
 3.1335577964782715]

In [35]:
robo.getl()

[-0.6497162204069298,
 -0.13795809657884417,
 -0.023874797606461887,
 -1.1959392473430532,
 -1.2375635231650854,
 1.206342782608168]

In [42]:
robo.get_orientation()

<Orientation: 
array([[ 1.27009257e-03,  6.29905468e-04, -9.99998995e-01],
       [ 9.99997712e-01, -1.72197259e-03,  1.26900626e-03],
       [-1.72117151e-03, -9.99998319e-01, -6.32091092e-04]])>

In [ ]:


[0.009172797203063965,
 -0.8363850873759766,
 1.953855339680807,
 -1.1332073968699952,
 1.5791306495666504,
 3.1335577964782715]

In [24]:
robo.movej([0.009172797203063965,
 -0.8363850873759766,
 1.953855339680807,
 -1.1332073968699952,
 1.5791306495666504,
 3.1335577964782715],0.2,0.2,wait=False)

In [ ]:
[-0.6497162204069298,
 -0.13795809657884417,
 -0.023874797606461887,
 -1.1959392473430532,
 -1.2375635231650854,
 1.206342782608168]

In [ ]:
<Orientation: 
array([[-0.018508  ,  0.00970795, -0.99978158],
       [ 0.99970475,  0.01592528, -0.01835195],
       [ 0.01574364, -0.99982606, -0.00999983]])>

In [37]:
np.asarray([-645.1526295526423,-186.55543766435269,-23.841822046342515]) + (np.asarray([0.00970795,0.01592528,-0.99982606])*151.7)

array([-643.67993354, -184.13957269, -175.51543535])